In [114]:
import requests
import json

In [3]:
dialog_ids1 = [
    '5e412fabd655fa53fa6bc9f8',
    '5e41291ed655fa53fa6bc927',
    '5e412de7d655fa53fa6bc97c'
]

In [6]:
def get_dialog_url(dialog_id):
    return f"http://docker-externalloa-lofsuritnple-525614984.us-east-1.elb.amazonaws.com:4242/api/dialogs/{dialog_id}"

In [10]:
dialog_data = requests.get(get_dialog_url(dialog_ids1[0])).json()

In [31]:
(dialog_data['utterances'][0]['user']['user_type'], dialog_data['utterances'][0]['text'], 
 dialog_data['utterances'][0]['hypotheses'][0]['skill_name'], dialog_data['utterances'][0]['hypotheses'][0]['text'], 
 dialog_data['utterances'][0]['hypotheses'][0]['confidence'])

('human', "let's chat", 'dummy_skill', "Let's talk about something else.", 0.5)

In [39]:
dialog_data['utterances'][1]['user']['user_type'], dialog_data['utterances'][1]['text'], dialog_data['utterances'][1]['active_skill']

('bot',
 "Hi, this is an Alexa Prize Socialbot! Oscars are coming: '1917', Joaquin Phoenix, Renee Zellweger and other names expected to win big at the night! How are you?",
 'program_y')

In [70]:
len(dialog_data)

9

In [104]:
curr_ind = 0
utterances = dialog_data['utterances']
rows = []
for i in range(len(utterances)):
    if utterances[i]['user']['user_type'] == 'bot':
        curr_bot_utt = utterances[i]
        if i - 1 < 0:
            user_utts = []
            bot_utts = []
            new_utts = utterances[i:i]
        elif i - 2 < 0:
            user_utts = [{"text": utterances[i - 1]['text']}]
            bot_utts = []
            new_utts = utterances[i-1:i]
        elif i - 4 < 0:
            user_utts =[{"text": utterances[i - 1]['text']}, {"text": utterances[i - 3]['text']}]
            bot_utt = utterances[i-2]
            bot_utts = [{'text': bot_utt['text'], 'confidence': bot_utt['confidence'], 'skill_name': bot_utt['active_skill']}]
            new_utts = utterances[i-3:i]
        else:
            bot_utt = utterances[i-2]
            prev_bot_utt = utterances[i-4]
            user_utts = [{"text": utterances[i - 1]['text']}, {"text": utterances[i - 3]['text']}]
            bot_utts = [
                {'text': bot_utt['text'], 'confidence': bot_utt['confidence'], 'skill_name': bot_utt['active_skill']}, 
                {'text': prev_bot_utt['text'], 'confidence': prev_bot_utt['confidence'], 'skill_name': prev_bot_utt['active_skill']}
            ]
            new_utts = utterances[i-4:i]
        hypots = utterances[i - 1]['hypotheses']
                
        row = {'human_utterances': user_utts, 'bot_utterances': bot_utts, 'hypotheses': hypots, 'utterances': new_utts}
        rows.append(row)

In [108]:
dialogs_json = {
    "dialogs": 
    [
        {
            "human_utterances": rows[0]['human_utterances'],
            "bot_utterances": rows[0]['bot_utterances'],
            "utterances": rows[0]['utterances'],
            "human": {"profile": {}}
        }
    ]
}

In [116]:
with open('../response_selectors/convers_evaluation_based_selector/test_data.json', 'w') as f:
    json.dump(dialogs_json, f, indent=4, ensure_ascii=False)

In [110]:
result = requests.post("http://localhost:8009/respond", json=dialogs_json)

In [112]:
result.json()

[['program_y',
  "Hi, this is an Alexa Prize Socialbot! Oscars are coming: '1917', Joaquin Phoenix, Renee Zellweger and other names expected to win big at the night! How are you?",
  0.98,
  {},
  {}]]

In [128]:
for row in rows:
    for utt in row["utterances"]:
        print(f'{utt["user"]["user_type"]}: {utt["text"]}')
    for h in sorted(row["hypotheses"], key=lambda x: x["confidence"], reverse=True):
        if h['confidence'] > 0:
            print(f'hypot: {h["text"]}; skill: {h["skill_name"]}; conf: {h["confidence"]}')
    print()

human: let's chat
hypot: Hi, this is an Alexa Prize Socialbot! Oscars are coming: '1917', Joaquin Phoenix, Renee Zellweger and other names expected to win big at the night! How are you?; skill: program_y; conf: 0.98
hypot: Should it be permitted?; skill: alice; conf: 0.65
hypot: how are you doing tonight?; skill: dummy_skill; conf: 0.6
hypot: Let's talk about something else.; skill: dummy_skill; conf: 0.5

human: let's chat
bot: Hi, this is an Alexa Prize Socialbot! Oscars are coming: '1917', Joaquin Phoenix, Renee Zellweger and other names expected to win big at the night! How are you?
human: wait twenty seventeen
hypot: I'm waiting.  I am very logical and rational.; skill: alice; conf: 0.65
hypot: do you like to drive?; skill: dummy_skill; conf: 0.6
hypot: I'm waiting.; skill: program_y; conf: 0.6
hypot: I really do not know what to answer.; skill: dummy_skill; conf: 0.5
hypot: yes, were you aware there was a sequel to the first incredibles in video game form called , " the incredibl

In [ ]:
# cobot recommends to take 2 last utt for conversation evaluation service
# conv["pastUtterances"] = [uttr["text"] for uttr in dialog["human_utterances"]][-3:-1]
# conv["pastResponses"] = [uttr["text"] for uttr in dialog["bot_utterances"]][-2:]
# collect all the conversations variants to evaluate them batch-wise

In [ ]:
## Input:
# 2 past human utts
# 2 past bot utts
# current human utt
# new bot utts

## Output
# Score for each new bot utt